<a href="https://colab.research.google.com/github/DjordjeRadovanovic/PPPO_Projekat_IT39_2020/blob/main/PPPO_Projekat_IT39_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Principi prezentacije i prepoznavanja oblika 2023/2024**


**Đorđe Radovanović IT39/2020**

**Tema:** Klasifikacija lišća

## **Sadržaj**

1. Uvod
2. Tema
3. Učitavanje dataseta
4. Projekat
5. Zaključak
6. Literatura

## **1. Uvod**

## **2. Metodologija**

## **3. Učitavanje dataseta**

Prvi korak je učitavanje seta podataka koji će se koristiti prilikom treniranja modela. Da bi se to uradilo, neophodno je skinuti .zip fajl koji sadrži sve potrebne podatke, pomoću **wget** komande.

In [1]:
!wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1r4yy2ZIUGoyESydQzY39L-0s8ld7fuXr&confirm=xxx' -O Leaf_Classification.zip

--2024-06-16 20:44:05--  https://drive.usercontent.google.com/download?id=1r4yy2ZIUGoyESydQzY39L-0s8ld7fuXr&confirm=xxx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.175.132, 2404:6800:4003:c1c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.175.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35953027 (34M) [application/octet-stream]
Saving to: ‘Leaf_Classification.zip’

Leaf_Classification 100%[===================>]  34.29M  19.4MB/s    in 1.8s    

2024-06-16 20:44:08 (19.4 MB/s) - ‘Leaf_Classification.zip’ saved [35953027/35953027]



Kada je Leaf_Classification.zip fajl skinut, potrebno je izvršiti dekompresiju podataka. To radimo pomoću **unzip** komande.

In [2]:
!unzip Leaf_Classification.zip

Archive:  Leaf_Classification.zip
  inflating: images.zip              
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: train.csv.zip           


Možemo uočiti da se u Leaf_Classification.zip fajlu nalaze još 4 posebna .zip fajla: *images.zip*, *sample_submission.csv.zip*, *train.csv.zip* i *test.csv.zip*. Analizom sadržaja ovih fajlova utvrđeno je da neće trebati *sample_submission.csv.zip*. Stoga, ponovićemo **unzip**, ali samo za ostala tri .zip fajla.

In [3]:
!unzip -qq images.zip
!unzip -qq train.csv.zip
!unzip -qq test.csv.zip

Sada imamo sve podatke raspakovane i spremne za korišćenje u daljem radu. Sledeći korak je pisanje samog koda za pripremu podataka, generisanje, treniranje i testiranje modela.

## **4. Projekat**

Prva stvar koju je potrebno uraditi je importovanje svih neophodnih biblioteka za projekat. Potrebno je koristiti **pandas** biblioteku za Python, koja ima DataFrame dvodimenzionalnu strukturu podataka koju možemo koristiti za predstavljanje tabelarnih CSV podataka. Takođe je potrebno koristiti **NumPy** biblioteku za matematičke operacije nad prosleđenim podacima.

VGG16 model se preuzima iz **Keras** biblioteke, koja se koristi u svrhe mašinskog učenja. **Keras** biblioteka sadrži veliki broj modela, uključujuči i VGG16, koji će se koristiti. **Keras** koristi TensorFlow backend.

In [ ]:
# Importovanje neophodnih biblioteka
import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
import os
from PIL import Image
import matplotlib.pyplot as plt

Sledeći korak je učitavanje i sređivanje CSV podataka iz train.csv fajla. Učitavanje se vrši pomoću read_csv funkcije iz pandas paketa.

Takođe se vrši definisanje direktorijuma u kojem se nalaze slike neophodne za treniranje/testiranje modela.

In [ ]:
# Učitavanje podataka iz train.csv u train_df DataFrame
train_df = pd.read_csv('train.csv')

# Definisanje direktorijuma sa slikama
img_dir = 'images/'

Potrebno je učitati slike iz direktorijuma i ubaciti ij u NumPy niz. Definisana je funkcija **load_and_preprocess_image** koja uzima putanju slika, vrši promenu dimenzija u 224x224 (kako bi bile pogodne za VGG16 model), i vraća NumPy niz koji će se kasnije upotrebiti.

In [ ]:
# Učitavanje i predprocesiranje slika
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img_array = np.asarray(img)
    img_array = img_array / 255.0
    return img_array

Sledeći korak je kreiranje listi x_train i y_train, koje će se koristiti za treniranje modela.

* **x_train** - slike lišća
* **y_train** - klase lišća, odgovaraju slici iz x_train

**Postupak:** Kreiraju se dve liste, i za svaki red u train_df se čitaju podaci. Podatak o ID-ju se pretvara u putanju slike, koja se učitava preko gorenavedene funkcije i ubacuje u x_train, dok se za vrednost y_train uzima naziv klase lista (**species**) koji stoji pored.

In [ ]:
# Kreiranje listi sa podacima za treniranje modela
x_train = []
y_train = []

# Popunjavanje x_train i y_train listi
for index, row in train_df.iterrows():
    id = row['id']
    species = row['species']
    img_path = os.path.join(img_dir, f'{id}.jpg')

    # Učitavanje i sređivanje slike
    img = load_and_preprocess_image(img_path)

    # Dodavanje slike u x_train listu
    x_train.append(img)

    # Dodavanje klase u y_train listu
    y_train.append(species)

Sada kada su kreirani x_train i y_train podaci za treniranje, neophodno je proveriti da li su odgovarajućeg oblika za model.

In [ ]:
# Provera x_train i y_train
print('x_train[0] shape:', x_train[0].shape)
print('y_train[0]:', y_train[0])

Kako podaci za x_train nisu odgovarajućeg oblika, neophodno je izvršiti konverziju. Potrebno je pretvoriti podatke u **NumPy** niz, i dodati četvrtu dimenziju - color channels. Kako RGB ima tri kanala, dodaje se broj 3.

In [ ]:
x_new = []
for image in x_train:
    img_array = np.asarray(image)
    x_new.append(img_array)
x_train = np.array(x_new)

# Konverzija x_train u odgovarajući oblik - iz (990, 224, 224) u (990, 224, 224, 3)
x_train = np.repeat(x_train[..., np.newaxis], 3, axis=-1)

Podaci iz y_train se moraju pretvoriti u numeričke vrednosti kako bi se moglo izvršiti treniranje modela. Ta konverzija se vrši pomoću LabelEncoder funkcije iz biblioteke sklearn.

LabelEncoder dodeljuje numeričku vrednost svakom nazivu klase, po abecednom redu.

U ovom koraku takođe se vrši sortiranje naziva klasa po abecedi, kako bi se kasnije moglo vršiti mapiranje broja klase sa najvećom verovatnoćom sa nazivom klase.

In [ ]:
# Transformacija labela u numeričke vrednosti, za output
classes_names = sorted(set(y_train)) #Da bismo sačuvali nazive klasa
y_train = LabelEncoder().fit_transform(y_train)
y_train = np.array(y_train)

Još jednom se vrši provera oblika input i output podataka za model.

In [ ]:
# Prikaz oblika x_train i y_train
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

Kada su podaci odgovarajućeg oblika, može se početi sa izgradnjom i treniranjem modela. Prvo je potrebno definisati ukupan broj mogućih klasa. U slučaju predikcije listova, ima ih 99.

In [ ]:
# Dobijanje ukupnog broja klasa
num_classes = train_df['species'].unique().size
print('Broj klasa: ', num_classes)

Koristiće se tehnika transfernog učenja (*transfer learning*).

**Ideja:** Koristi se postojeći model koji je već treniran za neki dataset. Takav model već ima razvijene tehnike uočavanja karakteristika koje se ne bi mogle razviti sa vrlo malim datasetom (kakav je ovaj). Postojeći model možemo prilagoditi novom setu podataka pomoću dodavanja slojeva, promene konfiguracije i ponovnog treniranja određenog broja slojeva sa novim datasetom.

U ovom slučaju koristiće se **VGG16** model kao osnova, jer sadrži dovoljan broj slojeva i dovoljno dobre performanse za ovaj slučaj analize.

Vrši se učitavanje baznog **VGG16** modela, konfiguracija njegovih slojeva. U ovom slučaju, potrebno je onemogućiti treniranje nekih slojeva tog modela.

*(Eksperimentisanjem je utvrđeno da je najbolje ostaviti poslednja dva modela otvorenim za treniranje, a svim drugim modelima zabraniti.)*

In [ ]:
# Konfiguracija VGG16
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Onemogućavanje treniranja slojeva VGG16 modela
for layer in vgg_conv.layers[:-2]:
    layer.trainable = False

for layer in vgg_conv.layers:
    print(layer, layer.trainable)

Sledeći korak je kreiranje modela. Koristi se sekvencijalni (Sequential) model, kome se dodaju slojevi. Kao početni sloj stavljaju se slojevi prethodno konfigurisanog VGG16 modela.

Pored njega, dodaju se još četiri sloja.

(Eksperimentisanjem je utvrđeno da ova četiri sloja daju najbolje rezultate.)

In [ ]:
# Kreiranje modela - sekvencijalni
model = keras.Sequential()

# Dodavanje VGG16 slojeva u model
model.add(vgg_conv)

# Dodavanje novih slojeva u model
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Prikazivanje podataka o modelu koji je kreiran
model.summary()

In [ ]:
# Kompilacija modela
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Treniranje modela
history = model.fit(x_train, y_train, epochs=15, validation_split=0.2, batch_size=32)

In [ ]:
# Krive tačnosti i gubitaka
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

# Prikaz tačnosti treninga i validacije
# Prikaz gubitka treninga i validacije

plt.figure(figsize=(20,10))
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95, wspace=0.05, hspace=0)
plt.subplot(121)
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy', size=20)
plt.legend(prop={'size': 10})
plt.grid()
plt.subplot(122)
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss', size=20)
plt.legend(prop={'size': 10})
plt.grid()
plt.show()

In [ ]:
# Čuvanje modela
#model.save('models/model.keras')

In [ ]:
# Učitavanje podataka iz test.csv u test_df DataFrame
test_df = pd.read_csv('test.csv')

# Kreiranje x_test podataka za predikciju
x_test = []

# Popunjavanje x_test liste
for index, row in test_df.iterrows():
    id = int(row['id'])                               # bugfix
    img_path = os.path.join(img_dir, f'{id}.jpg')
    img = load_and_preprocess_image(img_path)
    x_test.append(img)

x_new = []
for image in x_test:
    img_array = np.asarray(image)
    x_new.append(img_array)
x_test = np.array(x_new)

# Konverzija x_test u odgovarajući oblik - iz (594, 224, 224) u (594, 224, 224, 3)
x_test = np.repeat(x_test[..., np.newaxis], 3, axis=-1)

# Prikaz oblika x_test
print('x_test shape:', x_test.shape)

# Klasifikacija novih slika
y_test = model.predict(x_test)

In [ ]:
# Verovatnoće klasa za prvu test sliku
print(y_test[0])

# Prva test slika
plt.imshow(x_test[0])
plt.title(classes_names[np.argmax(y_test[0])])
plt.axis('off')

## **5. Zaključak**

## **6. Literatura**